In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os, json, copy, pickle, requests, tarfile
from astropy.time import Time
from astropy.table import Table
from astropy.coordinates import SkyCoord, get_sun
from astropy.coordinates import Angle
from astropy.cosmology import Planck18 as cosmo
from astropy import constants
from astropy import units as u
from alerts import get_alerts
from style import output_folder, big_fontsize, base_width, base_height, dpi, CB_color_cycle, bandcols
import seaborn as sns
from astropy.time import Time

from tqdm import tqdm

from scipy.optimize import root
from scipy.interpolate import interp1d
from scipy.integrate import quad, fixed_quad, quadrature, romberg

from flarestack.cosmo import get_rate, define_cosmology_functions
plt.style.use('tableau-colorblind10')

No scratch directory has been set. Using home directory as default.


In [22]:
url = "https://gcn.gsfc.nasa.gov/gcn3/all_gcn_circulars.tar.gz"
res = requests.get(url, stream=True)

with open("data/gcns.tar.gz", 'wb') as f:
    f.write(res.raw.read())

In [30]:
with tarfile.open("data/gcns.tar.gz", "r:gz") as tar:
    mem1 = tar.extractfile(tar.getmembers()[0]).read()

In [31]:
mem1

b'George Djorgovski reports:                                           #000\n\nIRAS 18119-1342 and the possible new SGR\n\n  We obtained a spectrum of the IR source IRAS 18119-1342, mentioned in \n  IAUC 6744 by Henden et al. as a possible counterpart of the SGR 18.....\n  The spectrum is of the M-type, exremely red (as expected), and shows no\n  apparent emission lines, including H-alpha.  Our best guess is that it is\n  an obscured M supergiant, unrelated to the SGR.  There is no need for anyone\n  else to waste any more telescope time on this object, I think.  Forgot to\n  mention: the data were obtained at the Palomar 200-inch on 22 Sep 97 UT,\n  and the result can be cited as "Djorgovski et al., priv. comm."\n\n  Best regards, George Djorgovski (george@oracle.caltech.edu)\n\n\n\n\n////////////////////////////////////////////////////////////////////////////\nEditorial note from S.Barthelmy:  This messeage is about 8th issued\nto the entire GCN list in the last month.  These message

In [13]:
[l for l in res.text.split("\n") if l.startswith("SUBJECT")]

['SUBJECT: IceCube-220115A - IceCube observation of a high-energy neutrino candidate track-like event',
 'SUBJECT: IceCube Alert 220115.51: Global MASTER-Net observations report ',
 'SUBJECT: Fermi-LAT Gamma-ray Observations of IceCube-220115A and detection of a new gamma-ray source, Fermi J2350.2+2620',
 'SUBJECT: IceCube-220115A: Upper limits from a search for additional neutrino events in IceCube']

In [19]:
alerts = get_alerts()

In [42]:
txt = ""

gcn_circulars = pd.DataFrame(columns=["subjects"])

for n in alerts.Event.unique():
    url = f"https://gcn.gsfc.nasa.gov/other/icecube_{n[2:]}.gcn3"
    
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"{response.status_code} {response.text} for {n}")
        
    subjects = [l for l in response.text.split("\n") if l.startswith("SUBJECT")]
    gcn_circulars.loc[n] = [subjects]
    
gcn_circulars.to_csv("data/gcn_reports_on_observed_neutrinos.csv")

404 <!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>404 Not Found</title>
</head><body>
<h1>Not Found</h1>
<p>The requested URL /other/icecube_160427A.gcn3 was not found on this server.</p>
</body></html>
 for IC160427A
404 <!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>404 Not Found</title>
</head><body>
<h1>Not Found</h1>
<p>The requested URL /other/icecube_160731A.gcn3 was not found on this server.</p>
</body></html>
 for IC160731A
404 <!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>404 Not Found</title>
</head><body>
<h1>Not Found</h1>
<p>The requested URL /other/icecube_161210A.gcn3 was not found on this server.</p>
</body></html>
 for IC161210A
404 <!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>404 Not Found</title>
</head><body>
<h1>Not Found</h1>
<p>The requested URL /other/icecube_171028A.gcn3 was not found on this server.</p>
</body></html>
 for IC171028A
404 <!DOCTYPE HTML PUBLIC "-

In [50]:
for n, r in gcn_circulars.iterrows():
    txt += f"----- {n} ------\n{json.dumps(r.subjects, indent=4)}\n\n\n"
    
with open("data/gcn_reports_on_observed_neutrinos.txt", "w") as f:
    f.write(txt)

In [46]:
optical_telescopes = [
    "MASTER",
    "Pan-STARRS",
    "Lick",
    "KAIT",
    "Kanata",
    "GOTO",
    "Zwicky Transient Facility",
    "AbAO",
    "DECam",
    "DDOTI"
]

In [61]:
alerts["observed_by_others"] = False
alerts["observed_by"] = None

for n, r in gcn_circulars.iterrows():
    observed_by_mask = np.array([np.any([t in s for s in r.subjects]) for t in optical_telescopes])
    other_obs = np.any(observed_by_mask)
    alerts.loc[alerts.Event == n, "observed_by_others"] = other_obs
    alerts.loc[alerts.Event == n, "observed_by"] = json.dumps(list(np.array(optical_telescopes)[observed_by_mask]))

In [65]:
pd.set_option('display.max_rows', 500)
alerts[["Event", "observed_by_others", "observed_by"]][alerts.observed]

,Event,observed_by_others,observed_by
23,IC160427A,False,[]
24,IC160731A,False,[]
24,IC160731A,False,[]
26,IC160814A,False,[]
27,IC161103A,True,"[""MASTER""]"
28,IC161210A,False,[]
29,IC170312A,True,"[""MASTER""]"
30,IC170321A,False,[]
32,IC170922A,True,"[""MASTER""]"
35,IC171106A,True,"[""MASTER"", ""Pan-STARRS""]"


In [67]:
only_obserevd_by_asassn_m = alerts.observed & (~alerts.observed_by_others) & (~alerts.retracted)
only_observed_by_asassn_N = len(alerts[only_obserevd_by_asassn_m].Event.unique())
unretracted_alerts_N = len(alerts[~alerts.retracted].Event.unique()) - 2  # two have no localisation
fraction = only_observed_by_asassn_N / unretracted_alerts_N
print(f"{only_observed_by_asassn_N} out of {unretracted_alerts_N} alerts only obserevd by ASASSN: {fraction*100:.2f}%")

12 out of 71 alerts only obserevd by ASASSN: 16.90%
